# 2019년 1월 ~ 6월중 목요일에 지하철 승객 수가 많은가?

In [78]:
import os
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
%matplotlib inline 
import scipy.stats as stats
import numpy as np

In [68]:
os.listdir()
dirpath = './rawfiles/'
files = os.listdir(dirpath)
raw = pd.DataFrame()

for file in files:
    if file =='CARD_SUBWAY_MONTH_201907.csv':
        break
    else:
        temp = pd.read_csv(dirpath+file)
        raw = pd.concat([raw,temp],axis=0)
raw['사용일자'] = pd.to_datetime(raw['사용일자'], format='%Y%m%d')
raw['요일'] = raw['사용일자'].dt.dayofweek

In [69]:
conv = {
    0 : '월',
    1 : '화',
    2 : '수',
    3 : '목',
    4 : '금',
    5 : '토',
    6 : '일'
}
raw['요일'] = raw['요일'].map(conv)
raw["월"] = raw['사용일자'].dt.strftime('%m')      
raw['연월'] = raw['사용일자'].dt.strftime('%Y-%m')
raw['월일'] = raw['사용일자'].dt.strftime('%m-%d')
raw['승하자총승객수'] = raw['승차총승객수'] + raw['하차총승객수']
raw = raw[['사용일자','월','요일','노선명','역명','승차총승객수', '하차총승객수', '승하자총승객수','등록일자','연월','월일']]

In [103]:
raw

,사용일자,월,요일,노선명,역명,승차총승객수,하차총승객수,승하자총승객수,등록일자,연월,월일
0,2019-01-01,01,화,2호선,을지로4가,3862.0,3728.0,7590.0,20190104,2019-01,01-01
1,2019-01-01,01,화,3호선,을지로3가,8104.0,7554.0,15658.0,20190104,2019-01,01-01
2,2019-01-01,01,화,2호선,을지로입구,22478.0,21330.0,43808.0,20190104,2019-01,01-01
3,2019-01-01,01,화,2호선,시청,8381.0,6049.0,14430.0,20190104,2019-01,01-01
4,2019-01-01,01,화,6호선,동묘앞,8045.0,8504.0,16549.0,20190104,2019-01,01-01
...,...,...,...,...,...,...,...,...,...,...,...
17719,2019-06-30,06,일,7호선,마들,6622.0,6116.0,12738.0,20190703,2019-06,06-30
17720,2019-06-30,06,일,4호선,노원,14071.0,14702.0,28773.0,20190703,2019-06,06-30
17721,2019-06-30,06,일,7호선,중계,9516.0,8981.0,18497.0,20190703,2019-06,06-30
17722,2019-06-30,06,일,7호선,하계,11645.0,11121.0,22766.0,20190703,2019-06,06-30


In [71]:
b= pd.pivot_table(raw,index='요일',values='승하자총승객수',aggfunc='sum')
b

,승하자총승객수
요일,
금,445310717.0
목,428684383.0
수,421134136.0
월,391555551.0
일,247523995.0
토,341950018.0
화,411979965.0


# 목요일보다 금요일이 커서 가설 기각 그럼 금요일이 승객수가 차이가 있는가?

In [116]:
a=pd.DataFrame
a=raw[raw['요일']=='금']
a=a.groupby('사용일자')[['승하자총승객수']].sum()
a

,승하자총승객수
사용일자,
2019-01-04,16509281.0
2019-01-11,16673540.0
2019-01-18,16692844.0
2019-01-25,16846166.0
2019-02-01,15931852.0
2019-02-08,15795223.0
2019-02-15,17161453.0
2019-02-22,17192259.0
2019-03-01,11826597.0


# 금 데이터가 26개임으로 정규성을 검정할 필요가 있다.

# 정규성검정
### 귀무가설: dataset이 정규분포를 따른다
### 대립가설: dataset이 정규분포를 따르지 않는다
### p<0.05 이면 귀무가설을 기각하고 대립가설o
### p>0.05: 귀무가설 채택

In [73]:
type(a)

pandas.core.frame.DataFrame

In [74]:
test_stat,p_val = stats.shapiro(a)
print("{},p_value={}".format(test_stat,p_val))

0.7258557081222534,p_value=1.2227431398059707e-05


# p_value<0.05 임으로 귀무가설 즉 정규분포를 따르지 않는다
## 그럼으로 만위트니 검정을 사용한다 비모수검정


In [75]:
grouped_day = raw.groupby('요일')

group_mon = grouped_day.get_group('월')['승하자총승객수']
group_tue = grouped_day.get_group('화')['승하자총승객수']
group_wed = grouped_day.get_group('수')['승하자총승객수']
group_thu = grouped_day.get_group('목')['승하자총승객수']
group_fri = grouped_day.get_group('금')['승하자총승객수']
group_sat = grouped_day.get_group('토')['승하자총승객수']
group_sun = grouped_day.get_group('일')['승하자총승객수']


In [108]:
group_mon

3546     111363.0
3547      56003.0
3548      99517.0
3549      74850.0
3550      58726.0
           ...   
14173     61853.0
14174      9583.0
14175     21364.0
14176      4147.0
14177      7161.0
Name: 승하자총승객수, Length: 14790, dtype: float64

In [93]:
not_fri= pd.concat([group_mon, group_tue, group_wed, group_thu, group_sat, group_sun])

not_fri = not_thu.squeeze()
group_fri = group_thu.squeeze()


In [92]:
np.mean(not_fri)

24471.397453382942

In [94]:
np.mean(group_fri)

27869.222662852684

In [100]:
stats.mannwhitneyu(group_fri,not_fri)

MannwhitneyuResult(statistic=767322381.5, pvalue=2.1857073278968696e-69)

# 만위트니 검정(비모수 독립표본 검증 방식)으로 했을때 p<0.05임으로 금요일은 차이가있다.



In [114]:
grouped_Month = raw.groupby('월')

group_Jan = grouped_Month.get_group('01')['승하자총승객수']
group_Feb = grouped_Month.get_group('02')['승하자총승객수']
group_Mar = grouped_Month.get_group('03')['승하자총승객수']
group_Apr = grouped_Month.get_group('04')['승하자총승객수']
group_May = grouped_Month.get_group('05')['승하자총승객수']
group_Jun = grouped_Month.get_group('06')['승하자총승객수']

not_may= pd.concat([group_Jan,group_Feb,group_Mar,group_Jun,group_Apr])

In [115]:
stats.ttest_ind(group_May,not_may)

Ttest_indResult(statistic=7.768367505836027, pvalue=8.020550554084754e-15)

# p<0.05임으로 1~6월중 5월 지하철 승객수는 유의미한 차이가 있다.